# Animation 1 : WS2812B LED Control and Animation Loop Explanation

This document explains the functionality and implementation of the `main` loop for driving WS2812B LEDs using an STM32 microcontroller. It includes a detailed breakdown of the animation loop that cycles through LEDs with alternating colors, varying delays, and periodic resets.

## Table of Contents
1. [Introduction](#introduction)
2. [Setup and Configuration](#setup-and-configuration)
   - [MCU Initialization](#mcu-initialization)
   - [Peripherals Configuration](#peripherals-configuration)
3. [LED Control Loop](#led-control-loop)
   - [Overview](#overview)
   - [Color Definitions](#color-definitions)
   - [Cycling Logic](#cycling-logic)
   - [Delay Adjustments](#delay-adjustments)
4. [Code Walkthrough](#code-walkthrough)
   - [Key Functions](#key-functions)
   - [Infinite Loop Explanation](#infinite-loop-explanation)
5. [Conclusion](#conclusion)

---

## Introduction
WS2812B LEDs are addressable RGB LEDs widely used for creating colorful and dynamic lighting effects. This document focuses on the implementation of a control loop that:
- Cycles through all 16 LEDs in a strip.
- Alternates between shades of red, the current color, and shades of green.
- Adjusts the delay between cycles for a dynamic animation effect.

## Setup and Configuration
### MCU Initialization
The `HAL_Init()` function initializes the Hardware Abstraction Layer (HAL) to prepare the STM32 microcontroller for operation.

### Peripherals Configuration
1. **System Clock**: Configured using `SystemClock_Config()` for consistent timing.
2. **GPIO and DMA**: Required for WS2812B control using PWM signals.
3. **Timer Initialization**: Timer 3 is configured for PWM generation on a specific pin (e.g., PB5).

---

## LED Control Loop
### Overview
The control loop implements:
1. **Alternating Colors**: Shades of red, current color, and green in sequence.
2. **Dynamic Delays**: Gradual reduction in delay between cycles (1s, 0.5s, 0.25s).
3. **Full Reset**: Clearing all LEDs after each complete cycle.

### Color Definitions
Three distinct colors are defined in GRB format:
- `redShade`: A predefined shade of red.
- `currentColor`: The LED’s current color.
- `greenShade`: A predefined shade of green.

Example:
```c
uint8_t redShade[3] = {0x40, 0x00, 0x00};   // Shade of red
uint8_t currentColor[3] = {0x6B, 0x00, 0x3C}; // Current color
uint8_t greenShade[3] = {0x00, 0x40, 0x00}; // Shade of green
```

### Cycling Logic
The LEDs alternate colors based on their index position:
- LEDs with indices divisible by 3 are set to `redShade`.
- Indices with a remainder of 1 are set to `currentColor`.
- Indices with a remainder of 2 are set to `greenShade`.

The logic is implemented as:
```c
if (i % 3 == 0)
    WS2812_SetColor(i, redShade[0], redShade[1], redShade[2]);
else if (i % 3 == 1)
    WS2812_SetColor(i, currentColor[0], currentColor[1], currentColor[2]);
else
    WS2812_SetColor(i, greenShade[0], greenShade[1], greenShade[2]);
```

### Delay Adjustments
The delay between cycles decreases with each iteration:
1. Starts at 1 second.
2. Reduces to 0.5 seconds.
3. Further reduces to 0.25 seconds.

This is controlled using:
```c
for (float delay = 1.0; delay >= 0.25; delay /= 2)
{
    HAL_Delay((int)(delay * 1000)); // Delay in milliseconds
}
```

---

## Code Walkthrough
### Key Functions
1. **`WS2812_SetColor`**:
   Assigns color values to a specific LED in the buffer.
2. **`WS2812_Refresh`**:
   Converts buffer data into PWM signals and triggers DMA transfer to the LED strip.

### Infinite Loop Explanation
The `while(1)` loop performs the following steps:
1. Loops through the delays: 1s, 0.5s, and 0.25s.
2. Cycles through all 16 LEDs:
   - Assigns colors based on index.
   - Refreshes the LED strip.
   - Delays based on the current delay value.
3. Clears all LEDs after each cycle and adds a gap before the next cycle.

Example:
```c
for (int i = 0; i < NUM_LEDS; i++) {
    // Alternate colors
    if (i % 3 == 0)
        WS2812_SetColor(i, redShade[0], redShade[1], redShade[2]);
    else if (i % 3 == 1)
        WS2812_SetColor(i, currentColor[0], currentColor[1], currentColor[2]);
    else
        WS2812_SetColor(i, greenShade[0], greenShade[1], greenShade[2]);

    WS2812_Refresh();
    HAL_Delay((int)(delay * 1000));
}
```

---

## Conclusion
The WS2812B control loop leverages PWM and DMA for efficient LED control, alternating colors and delays to create dynamic lighting effects. This design ensures scalability and provides a template for more complex animations. By understanding each section of the loop, developers can adapt it for custom applications, enhancing creativity and functionality.

